In [1]:
import pandas as pd
import numpy as np
import ast

In [ ]:
#Este archivo JSON esta con comillas simples por lo que no se puede realizar una carga directa a pandas.

In [451]:
#Convierto el archivo json en una lista de python por lineas.
lineas_js = []
with open('./src/steam_games.json') as f:
    for line in f.readlines():
        lineas_js.append(ast.literal_eval(line))


df = pd.DataFrame(lineas_js)

In [52]:
# Elimino los registros que contienen precio nulo porque si el objetivo es predecir el precio considero que estos registros no me aportan al entrenamiento de mi modelo. Aprox 4% de los registros del dataset total.
#df = df.dropna(subset=['price'])

In [64]:
#Elimino las columnas url y reviews_url porque no las considero relevantes en este caso.
#df = df.drop(columns=['url', 'reviews_url'])

In [452]:
# Relleno los nulos en 'title' con los valores de 'app_name y elimino title para evitar redundancias.

df['title'] = df.apply(lambda row: row['app_name'] if pd.isnull(row['title']) else row['title'], axis=1)
df = df.drop(columns='app_name')

In [453]:
#Veo los nulos y borro el de indice 74 porque tiene practicamente toda la informacion NaN
df[df['title'].isnull()]


,publisher,genres,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
74,NaN,NaN,NaN,http://store.steampowered.com/,NaN,NaN,14.99,NaN,NaN,19.99,False,NaN,NaN,NaN,NaN
2580,NaN,"[Action, Indie]",NaN,http://store.steampowered.com/app/317160/_/,2014-08-26,"[Action, Indie]",NaN,http://steamcommunity.com/app/317160/reviews/?...,"[Single-player, Game demo]",NaN,False,317160,NaN,Positive,NaN


In [454]:
df = df.drop(74)

In [455]:
# Veo los precios que no son numericos (Free, Demo, etc) y los reemplazo por 0 para tener un valor numerico. Tambien reemplazo los valores 'Starting at x' por su version numerica.
df.price.unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 2.99, 3.99, 9.99,
       18.99, 29.99, nan, 'Free', 10.99, 1.59, 14.99, 1.99, 59.99, 8.99,
       6.99, 7.99, 39.99, 19.99, 7.49, 12.99, 5.99, 2.49, 15.99, 1.25,
       24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99,
       16.99, 79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99,
       149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 64.99,
       'Free Mod', 54.99, 74.99, 'Install Theme', 0.89, 'Third-party',
       0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0, 5.49, 23.99, 49.0, 20.99,
       10.93, 1.39, 'Free HITMAN™ Holiday Pack', 36.99, 4.49, 2.0, 4.0,
       9.0, 234.99, 1.95, 1.5, 199.0, 189.0, 6.66, 27.99, 10.49, 129.99,
       179.0, 26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33, 199.99,
       22.99, 320.0, 38.85, 71.7, 59.95, 995.0, 27.49, 3.39, 6.0, 19.95,
       499.99, 16.06, 4.68, 131

In [456]:
df['price'] = df['price'].replace(['Free To Play', 'Free to Play', 'Free', 'Free Demo', 'Play for Free!', 'Install Now', 'Play WARMACHINE: Tactics Demo', 'Free Mod', 'Install Theme', 'Third-party', 'Play Now', 'Free HITMAN™ Holiday Pack', 'Play the Demo', 'Free to Try', 'Free Movie', 'Free to Use'], 0.00)
df['price'] = df['price'].replace(['Starting at $499.00'], 499.00)
df['price'] = df['price'].replace(['Starting at $449.00'], 449.00)


In [457]:
df.release_date

0        2018-01-04
1        2018-01-04
2        2017-07-24
3        2017-12-07
4               NaN
            ...    
32130    2018-01-04
32131    2018-01-04
32132    2018-01-04
32133    2017-09-02
32134           NaN
Name: release_date, Length: 32134, dtype: object

In [458]:
from datetime import datetime

fecha_str = '2020-02-02'

# Convertir la fecha de texto a un objeto datetime
fecha_obj = datetime.strptime(fecha_str, '%Y-%m-%d')

# Obtener solo el año de la fecha
anio = fecha_obj.year

print(anio)

In [459]:
#Convierto release_date a Anio. Si el valor es NaN lo devuelvo como tipo fecha NaN: NaT
def convertir_a_anio(fecha_str):
    if pd.notnull(fecha_str):
        try:
            fecha_convertida = datetime.strptime(fecha_str, '%Y-%m-%d')
            anio = str(fecha_convertida.year)
            return anio
        except ValueError:
            return pd.NaT 
    else:
        return pd.NaT 

In [460]:

df['anio'] = df.release_date.apply(convertir_a_anio)

In [461]:
#Expando el dataframe original para ver los distintos generos y Creo un dataframe nuevo con las columnas Genero y Anio
df_generos = df.explode('genres')
columnas_a_mantener = ['genres', 'anio']
df_generos = df_generos.drop(columns=[col for col in df_generos.columns if col not in columnas_a_mantener])


In [462]:
#Expando el dataframe original para ver los distintos specs y Creo un dataframe nuevo con las columnas a Specs y Anio
df_specs = df.explode('specs')
columnas_a_mantener = ['specs', 'anio']
df_specs = df_specs.drop(columns=[col for col in df_specs.columns if col not in columnas_a_mantener])

In [463]:
lista_generos = df_generos.genres.unique()

In [464]:
lista_generos

array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', nan, 'Racing',
       'Early Access', 'Massively Multiplayer',
       'Animation &amp; Modeling', 'Video Production', 'Utilities',
       'Web Publishing', 'Education', 'Software Training',
       'Design &amp; Illustration', 'Audio Production', 'Photo Editing',
       'Accounting'], dtype=object)

In [465]:
#Lleno los datos NaN con un string No data en la columna sentiment para poder trabajar esa columna completa
df.sentiment.fillna("No data", inplace=True)

In [472]:
# Convierto los valores de string 'NA' en valores NaN con el parametro errors=coerce. Esto lo hago para poder tener solo numeros y NaN en la columna.
df['metascore'] = pd.to_numeric(df['metascore'], errors='coerce')

In [547]:
def genero(anio):
    
    df_gen_anio = df_generos[df_generos.anio == anio]
    lista_generos = []
    for _ in range (5):
        mayor = df_gen_anio.genres.describe().top
        lista_generos.append(mayor)
        df_gen_anio =  df_gen_anio[df_gen_anio['genres'] != mayor]
        
    return {anio:lista_generos}
    

In [544]:
def juegos(anio):
    df_juegos = df[df['anio'] == anio]
    return {anio: list(df_juegos.title)}
    

In [541]:
def specs(anio):
    df_specs_anio = df_specs[df_specs.anio == anio]
    lista_specs = []
    for _ in range (5):
        mayor = df_specs_anio.specs.describe().top
        lista_specs.append(mayor)
        df_specs_anio =  df_specs_anio[df_specs_anio['specs'] != mayor]
        
    return {anio:lista_specs}

In [537]:
def earlyaccess(anio):
    df_early = df[df['anio'] == anio]
    return {anio:df_early[df_early.early_access == True].title.count()}
    
    

In [402]:
def sentiment(anio):
    df_sent = df[df['anio'] == anio]
    dicc = {}
    mayor = ''
    cantidad = 0
    for i in range (len(df_sent.sentiment.unique())):
        if df_sent.sentiment.describe().top == 'No data':
            df_sent = df_sent[df_sent['sentiment'] != 'No data']
        else:
            mayor = df_sent.sentiment.describe().top
            cantidad = df_sent.sentiment.describe().freq
            dicc[mayor] = cantidad
            df_sent = df_sent[df_sent['sentiment'] != mayor]
    
    return dicc

In [523]:
def metascore(anio):
    df_meta = df[df['anio'] == anio]
    dicc_mayores = {}

    df_meta_mayor = df_meta.sort_values(by="metascore", ascending=False)
    for _, fila in df_meta_mayor.iterrows():
        if len(dicc_mayores) == 5 or df_meta_mayor.metascore.count() == len(dicc_mayores):
            break
        else:
            dicc_mayores[fila['title']] = fila['metascore']
    if len(dicc_mayores) == 0:
        return "No hay valoraciones registradas este anio"
    
    return dicc_mayores
    